In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import keras, os
import tensorflow as tf
from keras.models import Model
import matplotlib.pyplot as plt
from keras.models import Sequential
from tensorflow.keras.layers import concatenate
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import datasets, layers, models, losses
from keras.layers import Input, Dense, Conv2D, MaxPool2D , Flatten, MaxPooling2D, GlobalAveragePooling2D ,AveragePooling2D, Dropout, Activation, BatchNormalization
from keras.models import load_model
from keras_preprocessing.image import load_img
from keras_preprocessing.image import img_to_array
from tensorflow.keras.applications.vgg16 import VGG16
import numpy as np
import cv2
import os

# **Read Data**

In [ ]:
TRAIN_AUG_DIR = r"/content/drive/MyDrive/Sports Datasets/Train"
VALIDATION_AUG_DIR = r"/content/drive/MyDrive/Sports Datasets/Validation"

image_size=(224,224)
train_d = tf.keras.preprocessing.image_dataset_from_directory(
     TRAIN_AUG_DIR,
     labels="inferred",
     label_mode="categorical",
     color_mode="rgb",
     batch_size=64,
     image_size=image_size,
     seed=111,
     shuffle=True)

valid_d =tf.keras.preprocessing.image_dataset_from_directory(
    VALIDATION_AUG_DIR,
    labels="inferred",
    label_mode="categorical",
    color_mode="rgb",
    batch_size=64,
    image_size=image_size,
    seed=111,
    shuffle=True)

Found 14262 files belonging to 6 classes.
Found 4274 files belonging to 6 classes.


# **ResNet-50 _ Model**

In [ ]:
from tensorflow.keras.applications import ResNet50
base_model = ResNet50(input_shape=(224, 224,3), include_top=False, weights="imagenet")

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
x = layers.Flatten()(base_model.output)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(6, activation='Softmax')(x)
model = tf.keras.models.Model(base_model.input, x)
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['acc'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping 
early_stopping = EarlyStopping(restore_best_weights=True, patience=1)

In [ ]:
resnet_history = model.fit(train_d, epochs=20, validation_data=valid_d, callbacks = [early_stopping])

In [ ]:
def history_plot(history):
  acc = history.history['acc']
  val_acc = history.history['val_acc']
  loss = history.history['loss']
  val_loss = history.history['val_loss']
  epochs_range = range(len(acc))
  plt.figure(figsize=(12, 8))
  plt.subplot(2, 1, 1)
  plt.plot(epochs_range, acc, label='Training Accuracy')
  plt.plot(epochs_range, val_acc, label='Validation Accuracy')
  plt.legend(loc='lower right')
  plt.title('Training and Validation Accuracy')
  plt.subplot(2, 1, 2)
  plt.plot(epochs_range, loss, label='Training Loss')
  plt.plot(epochs_range, val_loss, label='Validation Loss')
  plt.legend(loc='upper right')
  plt.title('Training and Validation Loss')
  plt.show()

In [ ]:
history_plot(resnet_history)

# **Save Model**

In [ ]:
from tensorflow.keras.models import load_model
model.save('model_ResNet.h5')

In [ ]:
from keras.models import load_model
ResNet_model = load_model('model_ResNet.h5')